In [16]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib
warnings.filterwarnings('ignore')
np.set_printoptions(precision=3)
pd.options.display.float_format = '{:,.3f}'.format
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier # binary?
from sklearn.multiclass import OneVsOneClassifier  # multiclass
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC   # for titanic 73%,  RandomF 80%
from sklearn.svm import LinearSVC  #better for sparse input (NLP TfidfVectorizer)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import GridSearchCV
import tensorflow as tf


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dropout, LSTM, CuDNNLSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [3]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.array([x.flatten()/255.0 for x in X_train]) #normalized!
X_test = np.array([x.flatten()/255.0 for x in X_test]) 

In [4]:
X_test.shape

(10000, 784)

In [ ]:
#from sklearn import datasets
#iris=datasets.load_iris()
#X_train=iris.data 
#y_train=iris.target

In [10]:
def create_NN(neurons=1):
    model = Sequential()
    model.add(Dense(neurons, activation="relu", input_shape=(784,)))
    model.add(Dense(10, activation="softmax"))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

NN = KerasClassifier(build_fn=create_NN, verbose=0)

In [13]:
model_params={
    'logistic':{
        'model':LogisticRegression(),
        'params':{
        'solver':['lbfgs','liblinear'],
        'multi_class':['ovr']
        }    
    },    
    'svm':{
        'model':SVC(gamma='auto'),
        'params':{
            'C':[10,50],
            'kernel':['rbf']
        }
    },
    'RF':{
        'model':RandomForestClassifier(),
        'params':{
        'n_estimators':[50,100]
        }    
    },
    'kNN':{
        'model':KNeighborsClassifier(),
        'params':{
        'n_neighbors':[3,5]
        }
    },
    'Bayes':{
        'model':MultinomialNB(),
        'params':{
        'alpha':[1]
        }
    },
    'NN':{
        'model':NN,
        'params':{
            'epochs':[20],
            'batch_size':[64],
            'neurons':[10,20]
        }
    }
}

In [14]:
result=[]

for name, mp in model_params.items():
    t0=time.time()
    clf=GridSearchCV(mp['model'],mp['params'],cv=4,return_train_score=False, n_jobs=1)
    clf.fit(X_test,y_test)   #  smaller test set!!!
    t1=time.time()
    result.append({
        'model':name,
        'best_score':clf.best_score_,
        'best_params':clf.best_params_,
        'time':t1-t0
    })



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
df=pd.DataFrame(result, columns=['model','best_score','best_params','time'])
df

,model,best_score,best_params,time
0,logistic,0.899,"{'multi_class': 'ovr', 'solver': 'liblinear'}",61.847
1,svm,0.941,"{'C': 50, 'kernel': 'rbf'}",143.180
2,RF,0.943,{'n_estimators': 100},23.677
3,kNN,0.942,{'n_neighbors': 5},186.270
4,Bayes,0.821,{'alpha': 1},0.344
5,NN,0.918,"{'batch_size': 64, 'epochs': 20, 'neurons': 20}",57.586
